In [4]:
from keras.models import load_model
import re
import requests
from bs4 import BeautifulSoup
import time, os
import urllib.request
from urllib.request import HTTPError
import numpy as np
from keras.preprocessing import image
import imageio
from tqdm import tqdm_notebook

model = load_model("aurora-model-0.2.h5")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [11]:
headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
def get_processed_image(img):
    baseurl = 'http://nordlys.nipr.ac.jp/acaurora/Syowa/'
    try:
        t_image = imageio.imread(baseurl+img[2:]).reshape(720, 480, 3)
        return np.expand_dims(t_image,axis=0)
    except HTTPError as e:
        return None
def day_predictions(date):
    site = 'http://nordlys.nipr.ac.jp/acaurora/Syowa/html/wrap.php?html='+str(date)+'.html'

    response = requests.get(site)

    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img', src = re.compile(r'20'))
    imgs = [img['src'] for img in img_tags if not 'kog' in img['src']]

    predictions = []
    
    for img in tqdm_notebook(imgs):
        img = img[:-5]+".jpg"
        test_image = get_processed_image(img)
        if test_image is not None:
            result = model.predict(test_image)
            predictions.append((result)[0][0])
        else:
            print (img+" not found")
    return predictions


if __name__ == __main__:
    pass

[0.0, 0.0, 0.0, 0.0, 0.0]